# Environment Setup Tutorial

This notebook demonstrates how to set up a TensorTrade trading environment with data fetching, exchanges, portfolios, and data feeds.

## 📚 Related Tutorials

Before diving in, you may want to review these tutorials:

| Tutorial | Description |
|----------|-------------|
| [Architecture Overview](../docs/tutorials/01-foundations/02-architecture.md) | How TensorTrade components fit together |
| [Trading Basics](../docs/tutorials/02-domains/track-a-trading-for-rl/01-trading-basics.md) | Essential trading concepts for RL practitioners |
| [OMS Deep Dive](../docs/tutorials/02-domains/track-a-trading-for-rl/02-oms-deep-dive.md) | Understanding exchanges, wallets, and portfolios |
| [Observers & Feeds](../docs/tutorials/03-components/03-observers-feeds.md) | Feature engineering and data pipeline |

---

## Setup Data Fetching

In [1]:
import ta

import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed, NameSpace
from tensortrade.oms.instruments import USD, BTC, ETH, LTC
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order

## Fetch Historical Data

In [2]:
cdd = CryptoDataDownload()

bitfinex_data = pd.concat([
    cdd.fetch("Bitfinex", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitfinex", "USD", "ETH", "1h").add_prefix("ETH:")
], axis=1)

bitstamp_data = pd.concat([
    cdd.fetch("Bitstamp", "USD", "BTC", "1h").add_prefix("BTC:"),
    cdd.fetch("Bitstamp", "USD", "LTC", "1h").add_prefix("LTC:")
], axis=1)

In [3]:
bitfinex_data.head()

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,ETH:date,ETH:unix,ETH:open,ETH:high,ETH:low,ETH:close,ETH:volume
0,2018-05-15 06:00:00,1526364000,8723.8,8793.0,8714.9,8739.0,8988053.53,2018-05-15 06:00:00,1.526364e+09,733.12,736.48,731.19,733.04,4246576.84
1,2018-05-15 07:00:00,1526367600,8739.0,8754.8,8719.3,8743.0,2288904.12,2018-05-15 07:00:00,1.526368e+09,733.04,735.99,731.70,734.64,2044880.32
2,2018-05-15 08:00:00,1526371200,8743.0,8743.1,8653.2,8723.7,8891773.14,2018-05-15 08:00:00,1.526371e+09,734.64,734.65,722.00,731.32,7891317.14
3,2018-05-15 09:00:00,1526374800,8723.7,8737.8,8701.2,8708.1,2054868.28,2018-05-15 09:00:00,1.526375e+09,731.32,732.00,728.44,728.44,2111099.12
4,2018-05-15 10:00:00,1526378400,8708.1,8855.7,8695.8,8784.4,17309722.58,2018-05-15 10:00:00,1.526378e+09,728.44,739.30,725.52,735.21,7197617.75


In [4]:
bitstamp_data.head()

,BTC:date,BTC:unix,BTC:open,BTC:high,BTC:low,BTC:close,BTC:volume,LTC:date,LTC:unix,LTC:open,LTC:high,LTC:low,LTC:close,LTC:volume
0,2018-05-15 06:00:00,1.526364e+09,8733.86,8796.68,8707.28,8740.99,559.93,2018-05-15 06:00:00,1526364000,147.20,148.70,147.01,147.20,1907.28
1,2018-05-15 07:00:00,1.526368e+09,8740.99,8766.00,8721.11,8739.00,273.58,2018-05-15 07:00:00,1526367600,147.20,148.02,146.80,147.50,557.94
2,2018-05-15 08:00:00,1.526371e+09,8739.00,8750.27,8660.53,8728.49,917.79,2018-05-15 08:00:00,1526371200,147.50,147.50,144.54,145.64,3606.38
3,2018-05-15 09:00:00,1.526375e+09,8728.49,8754.40,8701.35,8708.32,182.62,2018-05-15 09:00:00,1526374800,145.64,146.62,145.15,145.54,369.67
4,2018-05-15 10:00:00,1.526378e+09,8708.32,8865.00,8695.11,8795.90,1260.69,2018-05-15 10:00:00,1526378400,145.54,149.14,144.95,147.66,2755.48


## Define Exchanges

An exchange needs a name, an execution service, and streams of price data in order to function properly.

The setups supported right now are the simulated execution service using simulated or stochastic data. More execution services will be made available in the future, as well as price streams so that live data and execution can be supported.

In [5]:


bitfinex = Exchange("bitfinex", service=execute_order)(
    Stream.source(list(bitfinex_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitfinex_data['ETH:close']), dtype="float").rename("USD-ETH")
)

bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(bitstamp_data['BTC:close']), dtype="float").rename("USD-BTC"),
    Stream.source(list(bitstamp_data['LTC:close']), dtype="float").rename("USD-LTC")
)

Now that the exchanges have been defined we can define our features that we would like to include, excluding the prices we have provided for the exchanges.

## Define External Data Feed

Here we will define the feed to use whatever data you would like. From financial indicators to alternative datasets, they will all have to be defined and incorporated into the `DataFeed` provided to the environment.

In [6]:
# Add all features for bitstamp BTC & ETH
bitfinex_btc = bitfinex_data.loc[:, [name.startswith("BTC") for name in bitfinex_data.columns]]
bitfinex_eth = bitfinex_data.loc[:, [name.startswith("ETH") for name in bitfinex_data.columns]]

ta.add_all_ta_features(
    bitfinex_btc,
    colprefix="BTC:",
    **{k: "BTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)


with NameSpace("bitfinex"):
    bitfinex_streams = [
        Stream.source(list(bitfinex_btc[c]), dtype="float").rename(c) for c in bitfinex_btc.columns
    ]
    bitfinex_streams += [
        Stream.source(list(bitfinex_eth[c]), dtype="float").rename(c) for c in bitfinex_eth.columns
    ]
    

# Add all features for bitstamp BTC & LTC
bitstamp_btc = bitstamp_data.loc[:, [name.startswith("BTC") for name in bitstamp_data.columns]]  
bitstamp_ltc = bitstamp_data.loc[:, [name.startswith("LTC") for name in bitstamp_data.columns]]

ta.add_all_ta_features(
    bitstamp_ltc,
    colprefix="LTC:",
    **{k: "LTC:" + k for k in ['open', 'high', 'low', 'close', 'volume']}
)

with NameSpace("bitstamp"):
    bitstamp_streams = [
        Stream.source(list(bitstamp_btc[c]), dtype="float").rename(c) for c in bitstamp_btc.columns
    ]
    bitstamp_streams += [
        Stream.source(list(bitstamp_ltc[c]), dtype="float").rename(c) for c in bitstamp_ltc.columns
    ]


feed = DataFeed(bitfinex_streams + bitstamp_streams)

/usr/local/lib/python3.12/site-packages/ta/wrapper.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_adi"] = AccDistIndexIndicator(
/usr/local/lib/python3.12/site-packages/ta/wrapper.py:96: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f"{colprefix}volume_obv"] = OnBalanceVolumeIndicator(
/usr/local/lib/python3.12/site-packages/ta/wrapper.py:101: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [7]:
feed.next()

{'bitfinex:/BTC:date': Timestamp('2018-05-15 06:00:00'),
 'bitfinex:/BTC:unix': 1526364000,
 'bitfinex:/BTC:open': 8723.8,
 'bitfinex:/BTC:high': 8793.0,
 'bitfinex:/BTC:low': 8714.9,
 'bitfinex:/BTC:close': 8739.0,
 'bitfinex:/BTC:volume': 8988053.53,
 'bitfinex:/BTC:volume_adi': -3441008.969871901,
 'bitfinex:/BTC:volume_obv': 8988053.53,
 'bitfinex:/BTC:volume_cmf': nan,
 'bitfinex:/BTC:volume_fi': nan,
 'bitfinex:/BTC:volume_em': nan,
 'bitfinex:/BTC:volume_sma_em': nan,
 'bitfinex:/BTC:volume_vpt': nan,
 'bitfinex:/BTC:volume_vwap': nan,
 'bitfinex:/BTC:volume_mfi': nan,
 'bitfinex:/BTC:volume_nvi': 1000.0,
 'bitfinex:/BTC:volatility_bbm': nan,
 'bitfinex:/BTC:volatility_bbh': nan,
 'bitfinex:/BTC:volatility_bbl': nan,
 'bitfinex:/BTC:volatility_bbw': nan,
 'bitfinex:/BTC:volatility_bbp': nan,
 'bitfinex:/BTC:volatility_bbhi': 0.0,
 'bitfinex:/BTC:volatility_bbli': 0.0,
 'bitfinex:/BTC:volatility_kcc': nan,
 'bitfinex:/BTC:volatility_kch': 8827.066666666668,
 'bitfinex:/BTC:volati

## Portfolio

Make the portfolio using the any combinations of exchanges and intruments that the exchange supports

In [8]:
portfolio = Portfolio(USD, [
    Wallet(bitfinex, 10000 * USD),
    Wallet(bitfinex, 10 * BTC),
    Wallet(bitfinex, 5 * ETH),
    Wallet(bitstamp, 1000 * USD),
    Wallet(bitstamp, 5 * BTC),
    Wallet(bitstamp, 3 * LTC),
])

## Environment

In [9]:
env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="simple",
    feed=feed,
    window_size=15,
    enable_logger=False
)

In [10]:
env.observer.feed.next()

{'internal': {'bitfinex:/USD-BTC': 8739.0,
  'bitfinex:/USD-ETH': 733.04,
  'bitfinex:/USD:/free': 10000.0,
  'bitfinex:/USD:/locked': 0.0,
  'bitfinex:/USD:/total': 10000.0,
  'bitfinex:/BTC:/free': 10.0,
  'bitfinex:/BTC:/locked': 0.0,
  'bitfinex:/BTC:/total': 10.0,
  'bitfinex:/BTC:/worth': 87390.0,
  'bitfinex:/ETH:/free': 5.0,
  'bitfinex:/ETH:/locked': 0.0,
  'bitfinex:/ETH:/total': 5.0,
  'bitfinex:/ETH:/worth': 3665.2,
  'bitstamp:/USD-BTC': 8740.99,
  'bitstamp:/USD-LTC': 147.2,
  'bitstamp:/USD:/free': 1000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 1000.0,
  'bitstamp:/BTC:/free': 5.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 5.0,
  'bitstamp:/BTC:/worth': 43704.95,
  'bitstamp:/LTC:/free': 3.0,
  'bitstamp:/LTC:/locked': 0.0,
  'bitstamp:/LTC:/total': 3.0,
  'bitstamp:/LTC:/worth': 441.59999999999997,
  'net_worth': 146201.75},
 'external': {'bitfinex:/BTC:date': Timestamp('2018-05-15 06:00:00'),
  'bitfinex:/BTC:unix': 1526364000,
  'bit